## This is just Idea to generate text data with pretrained Image Captioning Model

- To improve your model with this method, you maybe have to prepare more vocab and more training

- If you think this is interesting, please upvote this notebook 😉

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
!git clone https://github.com/yunjey/pytorch-tutorial.git

In [ ]:
%cd pytorch-tutorial/tutorials/03-advanced/image_captioning/
!pip install -r requirements.txt

In [ ]:
!wget https://www.dropbox.com/s/ne0ixz5d58ccbbz/pretrained_model.zip .

In [ ]:
!unzip pretrained_model.zip

In [ ]:
!wget https://www.dropbox.com/s/26adb7y9m98uisa/vocap.zip .

In [ ]:
!mkdir models data

In [ ]:
!cp encoder-5-3000.pkl models/encoder-2-1000.ckpt

In [ ]:
!cp decoder-5-3000.pkl models/decoder-2-1000.ckpt

In [ ]:
!unzip vocap.zip -d data

In [ ]:
!ls ../../../../../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg

In [ ]:
!pip install pycocotools > /dev/null

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
import torch
from torchvision import transforms 
from build_vocab import Vocabulary
from model import EncoderCNN, DecoderRNN
from PIL import Image
 
 
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 
def load_image(image_path, transform=None):
    image = Image.open(image_path)
    image = image.resize([224, 224], Image.LANCZOS)
    
    if transform is not None:
        image = transform(image).unsqueeze(0)
    
    return image
 
def show_text(image_path):
    # Image preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])
    
    # Load vocabulary wrapper
    with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)
 
    # Build models
    encoder = EncoderCNN(embed_size).eval()  # eval mode (batchnorm uses moving mean/variance)
    decoder = DecoderRNN(embed_size, hidden_size, len(vocab), num_layers)
    encoder = encoder.to(device)
    decoder = decoder.to(device)
 
    # Load the trained model parameters
    encoder.load_state_dict(torch.load(encoder_path))
    decoder.load_state_dict(torch.load(decoder_path))
 
    # Prepare an image
    image = load_image(image_path, transform)
    image_tensor = image.to(device)
    
    # Generate an caption from the image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    sampled_ids = sampled_ids[0].cpu().numpy()          # (1, max_seq_length) -> (max_seq_length)
    
    # Convert word_ids to words
    sampled_caption = []
    for word_id in sampled_ids:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    sentence = ' '.join(sampled_caption)
    
    # Print out the image and the generated caption
    image = Image.open(image_path)
    plt.imshow(np.asarray(image))
    plt.show()
    print (sentence)


encoder_path ='models/encoder-2-1000.ckpt'
decoder_path ='models/decoder-2-1000.ckpt'
vocab_path ='data/vocab.pkl'
    
# Model parameters (should be same as paramters in train.py)
embed_size=256
hidden_size=512
num_layers=1
    
import glob
files = sorted(glob.glob('../../../../../input/petfinder-pawpularity-score/train/*.jpg'))
for i, image_path in enumerate (files):
    sentence = show_text(image_path)
    if i == 5:
        break

#### Wow it's noisy ... haha.

#### We need more vocab.  )^o^(

In [ ]:
train_df = pd.read_csv('../../../../../input/petfinder-pawpularity-score/train.csv')
train_df.head()

In [ ]:
def generate_text(image_path):
    # Image preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])
    
    # Load vocabulary wrapper
    with open(vocab_path, 'rb') as f:
        vocab = pickle.load(f)
 
    # Build models
    encoder = EncoderCNN(embed_size).eval()  # eval mode (batchnorm uses moving mean/variance)
    decoder = DecoderRNN(embed_size, hidden_size, len(vocab), num_layers)
    encoder = encoder.to(device)
    decoder = decoder.to(device)
 
    # Load the trained model parameters
    encoder.load_state_dict(torch.load(encoder_path))
    decoder.load_state_dict(torch.load(decoder_path))
 
    # Prepare an image
    image = load_image(image_path, transform)
    image_tensor = image.to(device)
    
    # Generate an caption from the image
    feature = encoder(image_tensor)
    sampled_ids = decoder.sample(feature)
    sampled_ids = sampled_ids[0].cpu().numpy()          # (1, max_seq_length) -> (max_seq_length)
    
    # Convert word_ids to words
    sampled_caption = []
    for word_id in sampled_ids:
        word = vocab.idx2word[word_id]
        sampled_caption.append(word)
        if word == '<end>':
            break
    sentence = ' '.join(sampled_caption)
    return sentence

In [ ]:
file_ids = []
sentences = []
files = sorted(glob.glob('../../../../../input/petfinder-pawpularity-score/train/*.jpg'))
for i, image_path in enumerate (files):
    file_id = image_path.split('/')[-1].split('.')[0]
    sentence = generate_text(image_path)[8:-6]
    
    file_ids.append(file_id)
    sentences.append(sentence)
        
res = pd.DataFrame()
res['Id'] = file_ids
res['Description'] = sentences

print(res.shape)
res.head()

In [ ]:
res.to_csv('../../../../../working/PetFinder_ImageCaptionedDescriptionDataset.csv', index=False)

In [ ]:
# EOF